In [12]:
import pandas as pd
import requests

In [13]:
API_KEY = "7215bf43b5de4fd2a9161700c44d5ee9"
BASE_URL = "https://api.football-data.org/v4/competitions/PL/standings"
headers = {"X-Auth-Token": API_KEY}
response = requests.get(BASE_URL, headers=headers)
data = response.json() 
print(data.keys()) 
standings_table = data['standings'][0]['table']

dict_keys(['filters', 'area', 'competition', 'season', 'standings'])


In [14]:
standings_table

[{'position': 1,
  'team': {'id': 57,
   'name': 'Arsenal FC',
   'shortName': 'Arsenal',
   'tla': 'ARS',
   'crest': 'https://crests.football-data.org/57.png'},
  'playedGames': 12,
  'form': None,
  'won': 9,
  'draw': 2,
  'lost': 1,
  'points': 29,
  'goalsFor': 24,
  'goalsAgainst': 6,
  'goalDifference': 18},
 {'position': 2,
  'team': {'id': 61,
   'name': 'Chelsea FC',
   'shortName': 'Chelsea',
   'tla': 'CHE',
   'crest': 'https://crests.football-data.org/61.png'},
  'playedGames': 12,
  'form': None,
  'won': 7,
  'draw': 2,
  'lost': 3,
  'points': 23,
  'goalsFor': 23,
  'goalsAgainst': 11,
  'goalDifference': 12},
 {'position': 3,
  'team': {'id': 65,
   'name': 'Manchester City FC',
   'shortName': 'Man City',
   'tla': 'MCI',
   'crest': 'https://crests.football-data.org/65.png'},
  'playedGames': 12,
  'form': None,
  'won': 7,
  'draw': 1,
  'lost': 4,
  'points': 22,
  'goalsFor': 24,
  'goalsAgainst': 10,
  'goalDifference': 14},
 {'position': 4,
  'team': {'id': 5

In [15]:
#Establishing SQL connection
import psycopg2
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="bolanlelove",
    dbname="epl"
)
cur = conn.cursor()


In [16]:
import json
from datetime import datetime
# Populate SQL table
for team in standings_table:
    cur.execute("""
        INSERT INTO standings (
            team_id, team_name, position, played_games, won, draw, lost, points,
            goals_for, goals_against, goal_difference, last_update
        )
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        ON CONFLICT (team_id) DO UPDATE SET
            position=EXCLUDED.position,
            played_games=EXCLUDED.played_games,
            won=EXCLUDED.won,
            draw=EXCLUDED.draw,
            lost=EXCLUDED.lost,
            points=EXCLUDED.points,
            goals_for=EXCLUDED.goals_for,
            goals_against=EXCLUDED.goals_against,
            goal_difference=EXCLUDED.goal_difference,
            last_update=EXCLUDED.last_update;
    """, (
        team['team']['id'],
        team['team']['name'],
        team['position'],
        team['playedGames'],
        team['won'],
        team['draw'],
        team['lost'],
        team['points'],
        team['goalsFor'],
        team['goalsAgainst'],
        team['goalDifference'],
        datetime.now()
    ))

conn.commit()
cur.close()
conn.close()